# COMP561 - HW3: HMM for genome annotation

## Question #1

In this question, we implement a version of the Viterbi algorithm to find the optimal path in a HMM model with 4 states, representing types of genomic information (Intergenomic, Start, Stop, coding portions). The HMM model is calibrated using the annotated genome of Vibrio cholerae, the bacteria that causes cholera.

We then use our implementation to predict genes in a bacterial genomic sequence: Vibrio Vulnificus.

## 1 - Input files
Let's first read and parse the 2 input files:

* Vibrio_cholerae.GFC_11.dna.toplevel.fa: the genome sequence of the bacteria
* Vibrio_cholerae.GFC_11.37.gff3.gz: the annotated genome

We will only consider the positive strand. We will also ignore overlaps.


In [1]:
# upload files in Google Colab from my Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 21.5 MB/s eta 0:00:00


In [5]:
#imports needed to read the data
import gzip
from Bio import SeqIO
#from Bio.Seq import Seq
import io
import numpy as np
import json

In [6]:
# Function that reads and parses the genome sequence

def read_genome_sequence(file_path):
    if file_path.endswith('.gz'):
       # if the fasta_file is compressed in gzip
      with gzip.open(file_path, 'rt') as handle:
        genome_records = [(record.id, str(record.seq)) for record in SeqIO.parse(handle, "fasta")]
    else:
       # Otherwise, read it directly as text
     with open(file_path, 'r') as handle:
        genome_records = [(record.id, str(record.seq)) for record in SeqIO.parse(handle, "fasta")]
    # Convert the list to array
    genome_array = np.array(genome_records, dtype=object)

    return genome_array


# Function that reads annotations and filters for CDS features
# the function only keeps CDS portions, and the forward strand (+)
def parse_annotations(file_path):
  cds_features = []  # list to save the CDS portions
  if file_path.endswith('.gz'):
    with gzip.open(file_path, "rt") as handle:
        for line in handle:
            if line.startswith("#"):
                continue  # Skip headers and comments
            fields = line.strip().split("\t")

            feature_type = fields[2]
            strand = fields[6]
            if feature_type == "CDS" and strand == "+":
                # Extract relevant information from GFF line
                seq_id = fields[0] # the contig id
                start = int(fields[3])  #start position of the gene
                end = int(fields[4]) # last nucleotide of the gene
                attributes = fields[8]

                # Append the relevant data as a tuple
                cds_features.append((seq_id, start, end, strand, attributes))
  else:
       # Otherwise, read it directly as text
     with open(file_path, 'r') as handle:
        for line in handle:
            if line.startswith("#"):
                continue  # Skip headers and comments
            fields = line.strip().split("\t")

            feature_type = fields[2]
            strand = fields[6]
            if feature_type == "CDS" and strand == "+":
                # Extract relevant information from GFF line
                seq_id = fields[0] # the contig id
                start = int(fields[3])  #start position of the gene
                end = int(fields[4]) # last nucleotide of the gene
                attributes = fields[8]

                # Append the relevant data as a tuple
                cds_features.append((seq_id, start, end, strand, attributes))

    # Convert the list to array
  cds_array = np.array(cds_features, dtype=object)
  return cds_array


Let's read the genome file and the annotated genome and save the relevant data in arrays

In [7]:
genome_file = '/content/drive/MyDrive/ColabNotebooks/COMP561/Vibrio_cholerae.GFC_11.dna.toplevel.fa'
annotation_file = '/content/drive/MyDrive/ColabNotebooks/COMP561/Vibrio_cholerae.GFC_11.37.gff3.gz'

# Read genome sequence
genome_record = read_genome_sequence(genome_file)
print(f"Number of contigs: {len(genome_record)}")
#print(f"Genome length: {len(genome_record)}")

# Parse and filter annotations for CDS features
cds_features = parse_annotations(annotation_file)
print(f"Total CDS features: {len(cds_features)}")

# Print a few CDS features as example
print(cds_features[:1])


Number of contigs: 151
Total CDS features: 1887
[['DN38.contig00001' 654 2138 '+'
  'ID=CDS:KFE17928;Parent=transcript:KFE17928;protein_id=KFE17928']]


In [8]:
print(cds_features.shape, genome_record.shape)

(1887, 5) (151, 2)


Let's display an example of cds_feature, and corresponding gene

In [9]:
contig = 'DN38.contig00001'
start = cds_features[np.where(cds_features[:,0] == contig)][0,1]
stop = cds_features[np.where(cds_features[:,0] == contig)][0,2]
print(cds_features[np.where(cds_features[:,0] == contig)][0,:])

seq = genome_record[np.where(genome_record[:, 0] == contig)][0,1]
#print gene length
print('gene length: ', stop - start)
#print first 30 nucleotides
print('first 30 nucleotides: ')
print(seq[start-1:start+30])
print('lasr 30 nucleotides: ')
print(seq[stop -30: stop+1])

['DN38.contig00001' 654 2138 '+'
 'ID=CDS:KFE17928;Parent=transcript:KFE17928;protein_id=KFE17928']
gene length:  1484
first 30 nucleotides: 
ATGAATGCGTTCAATAAATTAGTAAAACATT
lasr 30 nucleotides: 
GCTCATTTGAGTCAGCGCTATCTGAAGTAAT


Total length of the genome

In [10]:
gen_len = np.vectorize(len)(genome_record[:, 1])
total_length = np.sum(gen_len)
print(f"Total length of the genome: {total_length} nucleotides")

Total length of the genome: 4012216 nucleotides


Although relatively rare, some genes may overlap. For this exercise, we will ignore overlaps. Let's implement a function that remove overlapping genes from cds_features: if 2 genes overlap, we will keep only the gene that starts before the other.  

In [11]:
#remove overlapping CDS
def remove_overlap(cds_features):
  # Sort the CDS features by start position
  cds_features = cds_features[cds_features[:, 1].argsort()]
  # Create a mask to filter out overlapping features
  keep_mask = np.ones(cds_features.shape[0], dtype=bool)

  for i in range(len(cds_features) - 1):
    for j in range(i + 1, len(cds_features)):
      if cds_features[i,0] == cds_features[j,0]:  # Check if same contig
        if (cds_features[i,1] < cds_features[j,2] and  # Check for overlap: start position of gene i is before last position of gene j
                cds_features[i,2] > cds_features[j,1]): # and last position of gene i is after start position of gene j
                #remove (j) - i.e put the jth mask value to False
                keep_mask[j] = False

  # Filter the original array using the mask, to remove overlapping genes
  return cds_features[keep_mask]

In [12]:
cds = remove_overlap(cds_features)
print(f"Total CDS features after removing overlaps: {len(cds)}")

Total CDS features after removing overlaps: 1714


After removing the overlapping genes, the total number of genes is reduced from 1887 to 1714.

Compute Statistics based on Genome and CDS

Let's implement a function to compute the main statistics required to define our HMM model:
* average length of intergenic regions
* average length of genes
* average nucleotide frequency in intergenic regions
* average codon frequency in genes
* average Start codon frequency
* average Stop codon frequency

In [13]:
def compute_length(genome_array, cds_array):
    intergenic_lengths = []  # List to store lengths of intergenic regions
    cds_lengths = []  # List to store lengths of CDS regions
    nucl_counts = {}  # Dictionary to store nucleotide counts
    codon_mid_counts = {}  # Dictionary to store codon counts in the gene section
    codon_start_counts = {}  # Dictionary to store start codon counts in CDS
    codon_stop_counts = {}  # Dictionary to store STOP codon counts in CDS

    # Iterate over each contig in the genome array
    for contig_id, sequence in genome_array:
        # Filter CDS features for the current contig and sort by start position
        contig_cds = cds_array[cds_array[:, 0] == contig_id] # sub-array of all CDS in current contig
        contig_cds = contig_cds[contig_cds[:, 1].argsort()] # sorted by start position

        # if there are no CDS features for the contig : the entire contig is considered as intergenic region
        if contig_cds.shape[0] == 0:
            continue

        # Track the end of the previous CDS
        previous_end = contig_cds[0,1] - 1  # initially: first CDS start

        # Loop through each CDS feature in this contig to find intergenic regions
        for index in range(0, len(contig_cds)):
            # Get start and end of the current CDS
            start, end = contig_cds[index,1], contig_cds[index,2]
            cds_lengths.append(end - start+1)

            # Loop through each codon in the current CDS to count codons
            cds_sequence = sequence[start - 1:end]  # Extract the current gene sequence (start -1 to adjust for 0-based indexing)
            length = len(cds_sequence)
            start_codon = cds_sequence[:3] # Start codon = 1st 3 nucleotides
            #if start codon not in ATG, GTG TTG
            if start_codon not in ['ATG', 'GTG', 'TTG']:
              print(f"Start codon not in ATG, GTG, TTG: {start_codon}")
              #print contig , start end
              print(contig_cds[index,0],start,end)
            stop_codon = cds_sequence[-3:] # Stop Codon = last 3 nucleotides
            #update dictionary of start codons
            if start_codon not in codon_start_counts:
                codon_start_counts[start_codon] = 1
            else:
                codon_start_counts[start_codon] += 1
            #update dictionary of stop codons
            if stop_codon not in codon_stop_counts:
                codon_stop_counts[stop_codon] = 1
            else:
                codon_stop_counts[stop_codon] += 1
            # Update dictionary of gene region codons, by reading nucleotides 3 by 3
            for i in range(3, length-3, 3):
                codon = cds_sequence[i:i + 3]
                if codon not in codon_mid_counts:
                  codon_mid_counts[codon] = 1
                else:
                  codon_mid_counts[codon] += 1

            # Count nucleotides in the current intergenic region
            if previous_end < start - 1: #ignore intergenic seq before first gene of the contig and after last gene
              inter_seq = sequence[previous_end:start]
              intergenic_length = start - previous_end
              intergenic_lengths.append(intergenic_length)
              for nucl in inter_seq:
                nucl_counts[nucl] = nucl_counts.get(nucl, 0) + 1


            # Update previous_end to the current CDS end
            previous_end = end

    return np.array(intergenic_lengths, dtype=object), np.array(cds_lengths, dtype=object), nucl_counts, codon_mid_counts, codon_start_counts, codon_stop_counts


Let's compute the statistics on our data:

In [14]:
#compute average intergenic length
intergenic_lengths, cds_lengths, nucl_count, codon_counts, codon_start, codon_stop = compute_length(genome_record, cds)
#sort codon_counts by alphabetic order
codon_counts = dict(sorted(codon_counts.items()))
print(f"Average intergenic length: {intergenic_lengths.mean()}")
print(f"Average CDS length: {cds_lengths.mean()}")
print(f"Nucleotide counts: {nucl_count}")
print(f"Codon counts: {codon_counts}")
print(f"Start Codon counts: {codon_start}")
print(f"Stop Codon counts: {codon_stop}")

print(f"Total intergenic length: {intergenic_lengths.sum()}")
print(f"Total CDS length: {cds_lengths.sum()}")
#validate total number of nucleotides
print(f"Total number of nucleotides in intergenic regions: {sum(nucl_count.values())}")
#validate total number of codons in CDS
print(f"Total number of nucleotides in CDS: {sum(codon_counts.values())*3}")
#difference between total intergenic length and nucleotides counts
print(f"Difference between total intergenic length and nucleotides counts: {intergenic_lengths.sum() - sum(nucl_count.values())}")
#difference between total CDS length and codons counts
print(f"Difference between total CDS length and codons counts: {cds_lengths.sum() - (sum(codon_counts.values())+sum(codon_start.values())+sum(codon_stop.values()))*3}")
print(f"Total number of start codons: {sum(codon_start.values())}")
print(f"Total number of stop codons: {sum(codon_stop.values())}")


Average intergenic length: 1204.5984800506649
Average CDS length: 993.4498249708284
Nucleotide counts: {'T': 501083, 'C': 461487, 'A': 504626, 'G': 434865}
Codon counts: {'AAA': 20365, 'AAC': 11619, 'AAG': 7580, 'AAT': 10752, 'ACA': 4296, 'ACC': 11713, 'ACG': 6342, 'ACT': 7391, 'AGA': 1388, 'AGC': 7904, 'AGG': 452, 'AGT': 6420, 'ATA': 1920, 'ATC': 14418, 'ATG': 13362, 'ATT': 17475, 'CAA': 18728, 'CAC': 6092, 'CAG': 10345, 'CAT': 7173, 'CCA': 7278, 'CCC': 3251, 'CCG': 6073, 'CCT': 6360, 'CGA': 2888, 'CGC': 9962, 'CGG': 1530, 'CGT': 11496, 'CTA': 4921, 'CTC': 8177, 'CTG': 16480, 'CTT': 7098, 'GAA': 22305, 'GAC': 8249, 'GAG': 13641, 'GAT': 21118, 'GCA': 11005, 'GCC': 12568, 'GCG': 17229, 'GCT': 11897, 'GGA': 4166, 'GGC': 14372, 'GGG': 4771, 'GGT': 15452, 'GTA': 6257, 'GTC': 8218, 'GTG': 15954, 'GTT': 9435, 'TAC': 8128, 'TAT': 8778, 'TCA': 5941, 'TCC': 3320, 'TCG': 5154, 'TCT': 6375, 'TGC': 2331, 'TGG': 7321, 'TGT': 3261, 'TTA': 10756, 'TTC': 7858, 'TTG': 12576, 'TTT': 14478}
Start Codon c

We see that there are 3 types of Start codons: ATG, GTG, TTG.  ATG is the more frequent Start Codon.  

In [15]:
# print shape of intergenic length and cds_length
print(f"Shape of intergenic_lengths: {intergenic_lengths.shape}")
print(f"Shape of cds_lengths: {cds_lengths.shape}")

Shape of intergenic_lengths: (1579,)
Shape of cds_lengths: (1714,)


In [16]:
#analysis of start codons
start_codons = np.zeros(cds.shape[0], dtype=object)
i = 0
for contig_id, start in cds[:,0:2]:
  start = genome_record[np.where(genome_record[:, 0] == contig_id)][0,1][start-1:start+2]
  start_codons[i] = start
  i += 1

starts = np.unique(start_codons, return_counts=True)
print(f"Start codons: {starts}")


Start codons: (array(['ATG', 'GTG', 'TTG'], dtype=object), array([1526,  122,   66]))


## Question 1a) Statistics on the genome

In [17]:
# compute nucleotide freq in intergenic regions
nucl_freq = {}
for nucl in nucl_count:
  nucl_freq[nucl] = nucl_count[nucl]/sum(nucl_count.values())
print(f"Nucleotide frequency in intergenic regions: {nucl_freq}")
#validate it sums to 1
print(f"Sum of nucleotide frequency in intergenic regions: {sum(nucl_freq.values())}")

Nucleotide frequency in intergenic regions: {'T': 0.26344212935336986, 'C': 0.24262471077426012, 'A': 0.26530484563849427, 'G': 0.22862831423387578}
Sum of nucleotide frequency in intergenic regions: 1.0000000000000002


In [18]:
#compute the codon frequency table for CDS(middle)
codon_freq = {}
for codon in codon_counts:
  codon_freq[codon] = codon_counts[codon]/sum(codon_counts.values())
print(f"Codon frequency in CDS: {codon_freq}")
#validate sum = 1
print(f"Sum of codon frequency in CDS: {sum(codon_freq.values())}")

Codon frequency in CDS: {'AAA': 0.03609772353025632, 'AAC': 0.02059511169644234, 'AAG': 0.013435833260954724, 'AAT': 0.019058321797069285, 'ACA': 0.007614820539454023, 'ACC': 0.020761730209177136, 'ACG': 0.011241431997490087, 'ACT': 0.013100823698115615, 'AGA': 0.002460281868892501, 'AGC': 0.01401013536867891, 'AGG': 0.00080118689102263, 'AGT': 0.011379689912312576, 'ATA': 0.0034032717494766585, 'ATC': 0.02555644379372628, 'ATG': 0.02368464433151412, 'ATT': 0.030975090532346147, 'CAA': 0.033196079856353575, 'CAC': 0.010798297655110313, 'CAG': 0.018336899087675017, 'CAT': 0.012714410551560452, 'CCA': 0.012900526975359958, 'CCC': 0.005762518988306571, 'CCG': 0.010764619445089451, 'CCT': 0.011273337670141432, 'CGA': 0.00511908792317114, 'CGC': 0.017658017275149204, 'CGG': 0.0027119821753642122, 'CGT': 0.02037708959999149, 'CTA': 0.008722656395403455, 'CTC': 0.014494038070557623, 'CTG': 0.02921141584967465, 'CTT': 0.01258147024884652, 'GAA': 0.03953644602712337, 'GAC': 0.014621660761162997

In [19]:
#freq of start codons
start_freq = {}
for codon in codon_start:
  start_freq[codon] = codon_start[codon]/sum(codon_start.values())
print(f"Start codon frequency in CDS: {start_freq}")

# freq of stop codons
stop_freq = {}
for codon in codon_stop:
  stop_freq[codon] = codon_stop[codon]/sum(codon_stop.values())
print(f"Stop codon frequency in CDS: {stop_freq}")

Start codon frequency in CDS: {'ATG': 0.8903150525087514, 'GTG': 0.07117852975495916, 'TTG': 0.038506417736289385}
Stop codon frequency in CDS: {'TAA': 0.6534422403733956, 'TGA': 0.1837806301050175, 'TAG': 0.16277712952158693}


In [20]:
# create the state emission probabilities based on nucleotidesfrequencies computed for intergenic regions and frequencies of codons in CDS
emissions = {
    'Intergenic': nucl_freq,  # Single nucleotide emissions for intergenic regions
    'Start': start_freq,      # Codon emissions for start of CDS
    'Middle': codon_freq,     # Codon emissions for middle of CDS
    'Stop': stop_freq        # Codon emissions for end of CDS
}

int_start = 1/intergenic_lengths.mean()
int_int = 1 - int_start
mid_stop = 1/(cds_lengths.mean()/3)
mid_mid = 1 - mid_stop
# Transition probabilities
transitions = {
    'Intergenic': {'Intergenic': int_int, 'Start': int_start},  # Most likely stay in Intergenic, some chance to Start
    'Start': {'Middle': 1},  # Start always transitions to Middle
    'Middle': {'Middle': mid_mid, 'Stop': mid_stop},  # Middle likely stays in Middle, small chance to transition to Stop
    'Stop': {'Intergenic': 1}  # Stop always transitions back to Intergenic
}


In [21]:
#convert emission and transition probas into log
log_transitions = {
    state: {next_state: np.log(prob) if prob > 0 else -np.inf
            for next_state, prob in trans.items()}
    for state, trans in transitions.items()
}
log_emissions = {
    state: {obs: np.log(prob) if prob > 0 else -np.inf
            for obs, prob in obs_probs.items()}
    for state, obs_probs in emissions.items()
}

Save all input matrices (log of emission probas and log of transition probas) in a json file

In [22]:
# save the transitions and emissions probability matrices is json file
#change working dir to gdrive
chdir = '/content/drive/MyDrive/ColabNotebooks/COMP561'
import os
os.chdir(chdir)
print(os.getcwd())
with open('hmm_config.json', 'w') as file:
    json.dump({'transitions': log_transitions, 'emissions': log_emissions}, file)
    print("hmm_config.json saved")

/content/drive/MyDrive/ColabNotebooks/COMP561
hmm_config.json saved


In [ ]:
# save input matrices in a jason file


## Question 1b) Viterbi algorithm

In [55]:
# function to read the external parameters file
def load_hmm_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config['transitions'], config['emissions']

#Implementation of Viterbi algo
def viterbi(sequence, transitions, emissions):
    n = len(sequence)
    states = ['Intergenic', 'Start', 'Middle', 'Stop']
    #initialize the matrix V with -infty
    V = {state: np.full(n, -np.inf) for state in states}
    #initialize the backtrack matrix
    backtrack = {state: np.full(n, None) for state in states}

    # Initialize the first position
    V['Intergenic'][0] = 0  # log(1) = 0 since starting in Intergenic state

    # Fill the dynamic programming table
    for i in range(1, n):
        nucleotide = sequence[i]
        codon = sequence[i-3:i] if i >= 3 else None

        # Calculate log probabilities for each state
        for state in states:
            if state == 'Intergenic':
                max_prob, max_state = max(
                    (V[prev_state][i-1] + transitions[prev_state].get(state, -np.inf) + emissions[state].get(nucleotide, -np.inf), prev_state)
                    for prev_state in states
                )
                #update the V matrix with the max value
                V[state][i] = max_prob
                #update the backtrac matrix with the state leading to the max value
                backtrack[state][i] = max_state

            elif state == 'Start':
                if i >= 3:
                    max_prob, max_state = max(
                        (V[prev_state][i-3] + transitions[prev_state].get(state, -np.inf) + emissions[state].get(codon, -np.inf), prev_state)
                        for prev_state in states
                    )
                    #update the V matrix and backtrack matrix
                    V[state][i] = max_prob
                    backtrack[state][i] = max_state

            elif state == 'Middle':
                if i >= 3:
                    max_prob, max_state = max(
                        (V[prev_state][i-3] + transitions[prev_state].get(state, -np.inf) + emissions[state].get(codon, -np.inf), prev_state)
                        for prev_state in states
                    )
                    #update the V matrix and backtrack matrix
                    V[state][i] = max_prob
                    backtrack[state][i] = max_state

            elif state == 'Stop':
                if i >= 3:
                    max_prob, max_state = max(
                        (V[prev_state][i-3] + transitions[prev_state].get(state, -np.inf) + emissions[state].get(codon, -np.inf), prev_state)
                        for prev_state in states
                    )
                    #update the V matrix and backtrack matrix
                    V[state][i] = max_prob
                    backtrack[state][i] = max_state

    # Backtrack to determine the most likely path
    best_path = []
    # start with the max of the last column of matrix V
    max_final_state = max(states, key=lambda state: V[state][n-1])

    current_state = max_final_state

    i = n - 1
    # Backtrack the max proba states until beginning of sequence
    while i >= 0:
      if current_state == 'Intergenic':
         best_path.append((current_state, i))
         current_state = backtrack[current_state][i]
         i = i-1

      if current_state in ['Start', 'Middle', 'Stop']:
          for k in range(0,3):
            # add the current state in the 3 positions of the codon:
            best_path.append((current_state, i-k))
          current_state = backtrack[current_state][i]
          i = i-3
    # reverse the path to get a path from beginning to end of sequence
    best_path.reverse()
    assert(n==len(best_path))

    # Extract gene predictions from best_path
    genes = []
    gene_start = None

    for state, pos in best_path:
        if state == 'Start':
            gene_start = pos
        elif state == 'Stop' and gene_start is not None:
            genes.append((gene_start -2, pos + 2))  # Adjust to 1-based indexing and include last codon
            gene_start = None

    return genes

# function that predicts genes in a genome_file, and save the predicted genes in a gff3 format.
def predict_genes(genome_file, config_file, output_file):
    transitions, emissions = load_hmm_config(config_file)
    # read the genome file
    genome_record = read_genome_sequence(genome_file)
    #sort genome_record by contig
    genome_record = genome_record[genome_record[:, 0].argsort()]
    #open output file:
    with open(output_file, 'w') as out:
    # predict genes for each contig
      for contig_id, sequence in genome_record:
        genes = viterbi(sequence, transitions, emissions)
        # Write each gene prediction in GFF format
        for start, end in genes:
          out.write(f"{contig_id}\tena\tCDS\t{start}\t{end}\t.\t+\t0\t.\n")


In [37]:
output_file = 'predicted_genes_Vibrio_cholerae.gff3'
predict_genes(genome_file, 'hmm_config.json', output_file)

In [38]:
#Import the resulting annotated genome
output_file = 'predicted_genes_Vibrio_cholerae.gff3'
cds_predictions = parse_annotations(output_file)


In [31]:
# analysis of perfect matches, start only matches, stop only matches and no match
def matches(cds_predictions, cds_features):
  perfect_matches = 0
  start_matches = 0
  stop_matches = 0
  no_match = 0
  n = len(cds_predictions)

  for i in range(n):
    contig = cds_predictions[i,0]
    gene = cds_predictions[i,1:3]
    annotated_genes = cds_features[cds_features[:,0]==contig][:,1:3]
    m = len(annotated_genes)
    for j in range(m):
      if gene[0] == annotated_genes[j,0] and gene[1] == annotated_genes[j,1]:
        perfect_matches += 1
        break
      elif gene[0] == annotated_genes[j,0]:
        start_matches += 1
        break
      elif gene[1] == annotated_genes[j,1]:
        stop_matches += 1
        break
    no_match = n - perfect_matches - start_matches - stop_matches
  return perfect_matches, start_matches, stop_matches, no_match



In [32]:
perfect_match, start_match, stop_match, no_match = matches(cds_predictions, cds)
print(f"Perfect matches: {perfect_match}")
print(f"Start only matches: {start_match}")
print(f"Stop only matches: {stop_match}")
print(f"No match: {no_match}")

Perfect matches: 1246
Start only matches: 0
Stop only matches: 377
No match: 761


In [33]:
ratio_perfect_match = perfect_match/len(cds_predictions)
ratio_start_match = start_match/len(cds_predictions)
ratio_stop_match = stop_match/len(cds_predictions)
ratio_no_match = no_match/len(cds_predictions)
## Print ratios as percentages with 2 decimal places
print(f"Ratio of perfect matches: {ratio_perfect_match * 100:.2f}%")
print(f"Ratio of start only matches: {ratio_start_match * 100:.2f}%")
print(f"Ratio of stop only matches: {ratio_stop_match * 100:.2f}%")
print(f"Ratio of no match: {ratio_no_match * 100:.2f}%")

Ratio of perfect matches: 52.27%
Ratio of start only matches: 0.00%
Ratio of stop only matches: 15.81%
Ratio of no match: 31.92%


In [34]:
perfect_match, start_match, stop_match, no_match = matches(cds,cds_predictions)
print(f"Perfect matches: {perfect_match}")
print(f"Start only matches: {start_match}")
print(f"Stop only matches: {stop_match}")
print(f"No match: {no_match}")

Perfect matches: 1246
Start only matches: 0
Stop only matches: 377
No match: 91


In [35]:
ratio_perfect_match = perfect_match/len(cds)
ratio_start_match = start_match/len(cds)
ratio_stop_match = stop_match/len(cds)
ratio_no_match = no_match/len(cds)
# Print ratios as percentages with 2 decimal places
print(f"Ratio of perfect matches: {ratio_perfect_match * 100:.2f}%")
print(f"Ratio of start only matches: {ratio_start_match * 100:.2f}%")
print(f"Ratio of stop only matches: {ratio_stop_match * 100:.2f}%")
print(f"Ratio of no match: {ratio_no_match * 100:.2f}%")

Ratio of perfect matches: 72.70%
Ratio of start only matches: 0.00%
Ratio of stop only matches: 22.00%
Ratio of no match: 5.31%


## Question 1c) Test the model on a genome (Vibrio Vulnificus)

In [56]:
genome_file = '/content/drive/MyDrive/ColabNotebooks/COMP561/Vibrio_vulnificus.ASM74310v1.dna.toplevel.fa.gz'
#predict genes
output_file = 'predicted_genes_vibrio_vulnificus.gff3'
predict_genes(genome_file, 'hmm_config.json', output_file)


## Question 1d) Accuracy of predicted genes

In [40]:
annotation_file = '/content/drive/MyDrive/ColabNotebooks/COMP561/Vibrio_vulnificus.ASM74310v1.37.gff3.gz'
cds_features = parse_annotations(annotation_file)
#cds without overlap
cds = remove_overlap(cds_features)
cds_predictions = parse_annotations(output_file)


In [41]:
#compare predicted genes to annotated genes
perfect_match, start_match, stop_match, no_match = matches(cds_predictions, cds)

print(f"Perfect matches: {perfect_match}")
print(f"Start only matches: {start_match}")
print(f"Stop only matches: {stop_match}")
print(f"No match: {no_match}")

Perfect matches: 1469
Start only matches: 0
Stop only matches: 478
No match: 1035


In [42]:
n= len(cds_predictions)
ratio_perfect_match = perfect_match/n
ratio_start_match = start_match/n
ratio_stop_match = stop_match/n
ratio_no_match = no_match/n
## Print ratios as percentages with 2 decimal places
print(f"Ratio of perfect matches: {ratio_perfect_match * 100:.2f}%")
print(f"Ratio of start only matches: {ratio_start_match * 100:.2f}%")
print(f"Ratio of stop only matches: {ratio_stop_match * 100:.2f}%")
print(f"Ratio of no match: {ratio_no_match * 100:.2f}%")

Ratio of perfect matches: 49.26%
Ratio of start only matches: 0.00%
Ratio of stop only matches: 16.03%
Ratio of no match: 34.71%


In [43]:
#compare predicted genes to annotated genes
perfect_match, start_match, stop_match, no_match = matches(cds, cds_predictions)

print(f"Perfect matches: {perfect_match}")
print(f"Start only matches: {start_match}")
print(f"Stop only matches: {stop_match}")
print(f"No match: {no_match}")

Perfect matches: 1469
Start only matches: 0
Stop only matches: 478
No match: 266


In [44]:
n = len(cds)
ratio_perfect_match = perfect_match/n
ratio_start_match = start_match/n
ratio_stop_match = stop_match/n
ratio_no_match = no_match/n
## Print ratios as percentages with 2 decimal places
print(f"Ratio of perfect matches: {ratio_perfect_match * 100:.2f}%")
print(f"Ratio of start only matches: {ratio_start_match * 100:.2f}%")
print(f"Ratio of stop only matches: {ratio_stop_match * 100:.2f}%")
print(f"Ratio of no match: {ratio_no_match * 100:.2f}%")

Ratio of perfect matches: 66.38%
Ratio of start only matches: 0.00%
Ratio of stop only matches: 21.60%
Ratio of no match: 12.02%


In [45]:
#number of predicted genes
print(f"Number of predicted genes: {len(cds_predictions)}")
#number of annotated genes
print(f"Number of annotated genes: {len(cds)}")

Number of predicted genes: 2982
Number of annotated genes: 2213


##Question 1e) Undetected genes

In [46]:
# read genome
genome_file = '/content/drive/MyDrive/ColabNotebooks/COMP561/Vibrio_vulnificus.ASM74310v1.dna.toplevel.fa.gz'
genome_record = read_genome_sequence(genome_file)
#save genome_record in txt file
with open('genome_record_Vibrio_Vulnificus.txt', 'w') as file:
    for record in genome_record:
        file.write(f"{record[0]}\t{record[1]}\n")

In [47]:
#compute average intergenic length
intergenic_lengths, cds_lengths, nucl_count, codon_counts, codon_start, codon_stop = compute_length(genome_record, cds)
#sort codon_counts by alphabetic order
codon_counts = dict(sorted(codon_counts.items()))
print(f"Average intergenic length: {intergenic_lengths.mean()}")
print(f"Average CDS length: {cds_lengths.mean()}")
print(f"Nucleotide counts: {nucl_count}")
print(f"Codon counts: {codon_counts}")
print(f"Start Codon counts: {codon_start}")
print(f"Stop Codon counts: {codon_stop}")

print(f"Total intergenic length: {intergenic_lengths.sum()}")
print(f"Total CDS length: {cds_lengths.sum()}")
#validate total number of nucleotides
print(f"Total number of nucleotides in intergenic regions: {sum(nucl_count.values())}")
#validate total number of codons in CDS
print(f"Total number of nucleotides in CDS: {sum(codon_counts.values())*3}")
#difference between total intergenic length and nucleotides counts
print(f"Difference between total intergenic length and nucleotides counts: {intergenic_lengths.sum() - sum(nucl_count.values())}")
#difference between total CDS length and codons counts
print(f"Difference between total CDS length and codons counts: {cds_lengths.sum() - (sum(codon_counts.values())+sum(codon_start.values())+sum(codon_stop.values()))*3}")
print(f"Total number of start codons: {sum(codon_start.values())}")
print(f"Total number of stop codons: {sum(codon_stop.values())}")


Start codon not in ATG, GTG, TTG: CTA
contig_32 3 236
Start codon not in ATG, GTG, TTG: ATA
contig_32 47600 48322
Start codon not in ATG, GTG, TTG: CTA
contig_17 2 235
Start codon not in ATG, GTG, TTG: GTC
contig_9 1 237
Start codon not in ATG, GTG, TTG: ATT
contig_25 26916 27113
Start codon not in ATG, GTG, TTG: CAA
contig_33 2 232
Start codon not in ATG, GTG, TTG: ATA
contig_33 4131 5648
Start codon not in ATG, GTG, TTG: AAT
contig_30 1 231
Start codon not in ATG, GTG, TTG: CTA
contig_4 1 234
Start codon not in ATG, GTG, TTG: CTA
contig_36 2 235
Start codon not in ATG, GTG, TTG: ATA
contig_60 79710 80237
Start codon not in ATG, GTG, TTG: CTA
contig_15 3 236
Start codon not in ATG, GTG, TTG: AAT
contig_14 3 233
Start codon not in ATG, GTG, TTG: AAT
contig_5 1 231
Start codon not in ATG, GTG, TTG: TTC
contig_76 1 237
Start codon not in ATG, GTG, TTG: ATT
contig_76 24462 25946
Start codon not in ATG, GTG, TTG: CTA
contig_18 1 234
Start codon not in ATG, GTG, TTG: GTC
contig_27 1 237
Sta

In [48]:
#function that find the different types of matches and retrieve the corresponding genes
def find_matches(cds_predictions, cds_features, genome):
  perfect_matches = []
  start_matches = []
  stop_matches = []
  no_match = []
  n = len(cds_predictions)
  #sort cds_features by contig and by start position
  cds_features = cds_features[cds_features[:,0].argsort()]
  cds_features = cds_features[cds_features[:,1].argsort()]

  #print(n)
  for i in range(n):
    contig = cds_predictions[i,0]
    gene = cds_predictions[i,1:3]
    st= gene[0]
    end = gene[1]
    seq = genome[np.where(genome[:, 0] == contig)][0,1][st-1:end]
   # print(st,end, seq)
    annotated_genes = cds_features[cds_features[:,0]==contig][:,1:3]
    m = len(annotated_genes)
    status = 0
    min_st = 0
    min_end =0
    for j in range(m):
      an_st = annotated_genes[j,0]
      an_end = annotated_genes[j,1]
      if j<m-1:
        next_st = annotated_genes[j+1,0]
        next_end = annotated_genes[j+1,1]
      if j == 0:
        min_st = an_st
        min_end = an_end
      elif an_st < st:
        min_st = an_st
        min_end = an_end
      elif end < an_end:
        break
    #  print(an_st, an_end)
      if st == an_st and end == an_end:
        perfect_matches.append((contig, st, end, an_st, an_end, seq))
        status = 1
        break
      elif st == an_st:
        start_matches.append((contig, st, end, an_st, an_end, seq))
        status = 1
        break
      elif end == an_end:
        stop_matches.append((contig, st, end, an_st, an_end, seq))
        status = 1
        break
    if status == 0:
      no_match.append((contig, st, end, min_st, min_end, next_st, next_end, seq))
  return np.array(perfect_matches), np.array(start_matches), np.array(stop_matches), np.array(no_match)


In [49]:
perfect_matches, start_matches, stop_matches, no_match = find_matches(cds_predictions, cds, genome_record)

In [50]:
print(f"Number of perfect matches: {len(perfect_matches)}")
print(f"Number of start matches: {len(start_matches)}")
print(f"Number of stop matches: {len(stop_matches)}")
print(f"Number of no match: {len(no_match)}")

Number of perfect matches: 1469
Number of start matches: 0
Number of stop matches: 478
Number of no match: 1035


In [51]:
#analysis predicted genes that are not in annotated genome
print("predicted genes - no match in annotated genome: ")
print(no_match[:10])

predicted genes - no match in annotated genome: 
[['contig_1' '242' '1327' '1317' '1838' '7266' '7907'
  'GTGCTCGCGTTTTTGACCTTCGGTTTCAGTGTTCTCGGCACTTTTATTGTCCGCTCGGGGATTTTGACATCGGTCCATGCGTTTGCCGTGGATCCAACCAAAGGTATTGTGCTTTTGCTGGTCATGGCGTTCATTTTTTTACTCACTTTTGCGTTATTGATCCTCAAAAGCGATAGCATTCCCGCTAAAGCCATTACCCATTGGCTAAGTCGCCAATACCTTACGGTGGTGGCGATGGGACTGTTACTGATCGCAACCAGTACCGTGTTCCTTGGCACCTTCTACCCAATGATTTATGAGCTGTTGCGGCTTGGAAGCATCTCTGTGGGTGCACCGTATTTCAATACGATGATTGCGCCATTAAGCCTCTTAGGGCTGCTAGCAATGGGGTGGGGGCCGTTACTAAAATGGCAGCAAGGACTGTTTCAGAGCCGCAAACGCGTTTTGATGGAGTTTGTGCTGTCAGCGGTACTCGGTGCCATGCTCTATTTGCTGCAAGTGCAGGTATGGGCGCCAATGACTTTGTTGTTCTGGTGTATTACGACCTGGGTGATCGTCAGTCACCTTCGGCTATTATTTGTGATGCCACCGACAAAGAGGCGCCAAAAACTGCCAATGGTGCTTTCTCATATTGGTATTGCATTGGTGTGCCTCGGTGCGATGATGAATGCCCAACATTCGTTTGAACGAAATGTTCGTGTGGAACCTAACTCGTACCATCAGTTCGATGGCTTTTCCATTCGTTATCAAGGAATTGACTGGTCCATTGGACCAAACTACACCGCGGAGCGAGCGTCGGTCTCACTTATCCTTGCCGATAGACAATTTAAATTGTTACCCGAAAGACGGCATTACCCGGTGAGGGTCATGAACATGACAGAGCCCGCAATCAAG

Predicted genes that do not have match in the annotated genome tend to occur in long intergenic regions of the genome, that contains false Start codons. For instance, in contig_32, intergenic region [25793: 33513] contains several 'ATG', 'GTG', 'TTG' codons, that are mistaken for START codons in our Viterbi algo - as well as several pseudo STOP codons 'TGA' 'TAA' TAG'. therefore, the Viterbi algo identifies 5 genes in this intergenic region:
* positions [26108: 26332],
* positions [27923,28081],
* positions [28772,28894],
* positions [30392,30670],
* positions [32924,33502].

Analysis of annotated genes not found by the Viterbi algo

In [52]:
perfect_matches, start_matches, stop_matches, no_match = find_matches( cds, cds_predictions, genome_record)

In [53]:
print(f"Number of perfect matches: {len(perfect_matches)}")
print(f"Number of start matches: {len(start_matches)}")
print(f"Number of stop matches: {len(stop_matches)}")
print(f"Number of no match: {len(no_match)}")

Number of perfect matches: 1469
Number of start matches: 0
Number of stop matches: 478
Number of no match: 266


In [54]:
#analysis annotated genes that are not found by the algo
print("predicted genes - no match in annotated genome: ")
print(no_match[:10])

predicted genes - no match in annotated genome: 
[['contig_27' '1' '237' '305' '1051' '2246' '3178'
  'GTCCTAAATATCGAACTTTATTATCTGCCACCTTACAGTCCAAACCTCAACCCAATAGAGCGGTTATGGAAAGTAATGAATGAGAAGTCGAGGAACAATGTTTACTTCAAAAGTAAACGGGACTTCAAGGTGGCAATAGACCATTTTTTTGCTGTGACTCTTCCAGAGATCGCAGGCTCTTTGACATCTCGAATTAACGATAATTTTCAGGTTCTCAAGCCTGCATCTTCAAGTTGA']
 ['contig_362' '1' '380' '0' '0' '2246' '3178'
  'GTTGAAAAATATATTGCTCAAATAGCGGCTACAGTTGGTAATCAGCCCCACGGGAGAGCGATGCGAGATGGTTTTATCGAGGACATGCCATTTATTTTTGTTGGTAGTTTTATTTTAATTTTTGCTTTTACACCTTTTTCAGAAGGTACCACTAATACTTTTGGTCGAGTGTGGTTACATTTTGCTACTAAGCACTTCGATACCATTATGATGCACTTCAATATGTTGATGGGGATCATGACGATTTTTGTCTCGTTGGGGGTCGATTACAGTTTGGCGAAAGCTTACAAAATGGATGGCATCTCGAGTGCTGTGTTATCACTGATGTGCTTTTTGTTGGTGGCAGCACCAGCTAAGGAAGGCGCTTTATCGATGGCTCT']
 ['contig_95' '1' '237' '304' '783' '2974' '3882'
  'TTCCTAAATATCGAACTTTATTATCTGCCACCTTACAGTCCAAACCTCAACCCAATAGAGCGGTTATGGAAAGTAATGAATGAGAAGTCGAGGAACAATGTTTACTTCAAAAGTAAACGGGACTTCAAGGTGGCAATAGACCATTTTTTTGCTGTGACTCTTCCAGAGATC

Undetected genes seem to begin with unsual START Codons: 'GTC' 'GTT' 'TTC' 'CTA' 'GGG'. Because the Viterbi algo was calibrated based on only 3 start codons: 'ATG', 'GTG', 'TTG', it cannot detect these genes.